In [1]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage,AIMessageChunk
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain.schema.runnable import Runnable,RunnableLambda,RunnableSequence,RunnablePassthrough,RunnableParallel,RunnableBranch

In [2]:
load_dotenv()
# Access them
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
api_key = os.getenv("LANGCHAIN_API_KEY")
open_ai_api_key=os.getenv("open_ai_api_key")
# print("LangChain Endpoint:", endpoint)
# print("API Key:", api_key)

In [3]:
model=ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4-turbo",
    api_key=open_ai_api_key,
    max_tokens=50
)

In [4]:
positive_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Generate a thank you note for this feedback: {feedback}."),
])

negative_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Generate a response addressing this negative feedback: {feedback}."),
])

neutral_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Generate a request for more details for this neutral feedback: {feedback}."),
])

escalate_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Generate a message to escalate this feedback to a human agent: {feedback}."),
])


In [5]:
classfication_template=ChatPromptTemplate([
     ("system","You are a helpful assistant."),
     ("human","Classify the sentiment of this feedback as postive ,negtive ,neutral, or escalate:{(feedback)}."),
])

In [6]:
branches=RunnableBranch(
    (
    lambda x: "postive" in x,
    positive_feedback_template  | model | StrOutputParser()
    ),
    (
    lambda x: "negtive" in x,
    negative_feedback_template | model | StrOutputParser()
    ),
     (
    lambda x: "neutral" in x,
    
    neutral_feedback_template | model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser()

)

In [7]:
classfication_chain=classfication_template | model | StrOutputParser()

In [8]:
chain=classfication_chain | branches 

In [9]:
review="The product is excellent.It really enjoyed using it found it very helpfull."
#"The product is terrible.It broke after just one use and the quality is very poor."

In [10]:
result=chain.invoke({"feedback:review"})

In [11]:
print(result)

Subject: Request for Additional Information Regarding Your Feedback

Dear [Customer's Name],

Thank you for reaching out to us. We noticed that your recent message intended to provide feedback does not include the actual content of the feedback or review. To better assist you
